## Read Data

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
dataset_root = "./Seasons"
os.listdir(dataset_root)

['Summer.csv', 'Summer_2.csv', 'Winter.csv', 'winter_pred.csv']

In [3]:
summer_df = pd.read_csv(os.path.join(dataset_root, "Summer.csv"))
summer_df.head()

,Station,X,Y,2000,2001,2002,2003,2004,2005,2006,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,25470023,409854,5919495,0,62,37,40,15,34,64,...,70,96,43,63,59,25,24,72,22,7
1,25480025,422075,5923197,0,80,53,66,68,57,51,...,21,20,11,93,84,16,9,85,72,64
2,25480026,412183,5923933,0,49,36,54,76,63,38,...,34,63,72,40,41,8,10,64,90,48
3,25481088,421783,5925854,86,75,54,55,55,48,35,...,35,35,45,83,75,90,26,87,64,63
4,25490020,423475,5918019,25,30,55,23,27,28,26,...,66,57,76,60,41,50,23,67,41,17


In [4]:
winter_df = pd.read_csv(os.path.join(dataset_root, "Winter.csv"))
winter_df.head()

,Station,X,Y,2000,2001,2002,2003,2004,2005,2006,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,25470023,409854,5919495.00,0,41,64,29,2,82,50,...,13,38,53,87,79.68895,76.899734,79.902374,79.901855,77.901870,81.902435
1,25480025,422075,5923197.00,79,95,77,64,46,50,77,...,90,84,69,65,86.04448,85.913506,85.819040,85.757220,85.716736,85.689650
2,25480026,412183,5923933.00,60,30,71,47,37,19,67,...,57,24,59,52,80.27586,80.641300,80.807014,80.890015,81.691360,82.309420
3,25481088,421783,5925854.00,0,85,74,13,78,65,55,...,81,31,14,79,76.36980,76.244820,76.204470,76.193180,76.190180,76.189380
4,25490020,423475,5918019.55,27,28,50,28,21,15,93,...,35,66,35,52,72.73278,70.759600,69.111860,68.091095,67.498020,67.162834


## Preparing Data

Picking an example data row first and then load the whole dataset later

In [5]:
example = summer_df.iloc[4]
example

Station    25490020
X            423475
Y           5918019
2000             25
2001             30
2002             55
2003             23
2004             27
2005             28
2006             26
2007             12
2008             35
2009              9
2010             31
2011             66
2012             57
2013             76
2014             60
2015             41
2016             50
2017             23
2018             67
2019             41
2020             17
Name: 4, dtype: int64

In [6]:
# drop these columns, as we don't need them
example = example.drop(["Station", "X", "Y"])
example

2000    25
2001    30
2002    55
2003    23
2004    27
2005    28
2006    26
2007    12
2008    35
2009     9
2010    31
2011    66
2012    57
2013    76
2014    60
2015    41
2016    50
2017    23
2018    67
2019    41
2020    17
Name: 4, dtype: int64

In [7]:
# pick columns
winter_col = winter_df.columns[3:]

# summer has one extra
summer_col = summer_df.columns[3:]

In [8]:
summer_data = summer_df.loc[:, summer_col].values
winter_data = winter_df.loc[:, winter_col].values

In [9]:
summer_data

array([[ 0, 62, 37, ..., 72, 22,  7],
       [ 0, 80, 53, ..., 85, 72, 64],
       [ 0, 49, 36, ..., 64, 90, 48],
       ...,
       [54, 61, 87, ..., 50, 45, 33],
       [35, 60, 66, ..., 62, 53, 33],
       [48, 52, 93, ..., 94, 81, 83]], dtype=int64)

## Preprocessing

### Create sequences and labes

Last element of each data row is the label(target prediction) and the rest is input

In [10]:
def create_seq_label(data):
    seq = np.zeros(shape=(data.shape[0], data.shape[1] - 1))
    labels = np.zeros(shape=(data.shape[0], 1))
    
    for i in range(data.shape[0]):
        seq[i] = data[i][:-1]
        labels[i] = data[i][-1]

        
    return seq, labels

In [11]:
winter_seq, winter_labels = create_seq_label(winter_data)
summer_seq, summer_labels = create_seq_label(summer_data)

## Split data into train-test sets

Split ratio 50:50

In [12]:
from sklearn.model_selection import train_test_split

x_train_winter, x_test_winter, y_train_winter, y_test_winter = train_test_split(winter_seq, winter_labels, test_size=0.6)
x_train_summer, x_test_summer, y_train_summer, y_test_summer = train_test_split(summer_seq, summer_labels, test_size=0.6)

## LSTM

In [13]:
import torch
from torch import nn
import torch.nn.functional as F

In [14]:
if torch.cuda.is_available():
    device = torch.device(f"cuda:{torch.cuda.current_device()}")
    print(f"Setting Device to -> {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    device = torch.device("cpu")
    print("No gpu found, using cpu")

No gpu found, using cpu


#### Definition

In [15]:
class GrundwasserLSTM(nn.Module):
    def __init__(self, in_dim=1, hidden_dim=100, out_dim=1):
        super(GrundwasserLSTM, self).__init__()
        
        self.hidden_dim=hidden_dim
        
        self.lstm = nn.LSTM(in_dim, hidden_dim)
        self.linear = nn.Linear(hidden_dim, out_dim)
        
        # basically a square matrix
        self.hidden_cell = (torch.zeros(1,1,self.hidden_dim).to(device),
                            torch.zeros(1,1,self.hidden_dim).to(device))
        
    def forward(self, x):
        lstm_out, self.hidden_cell = self.lstm(x.view(len(x), 1, -1), self.hidden_cell)
        y = self.linear(lstm_out.view(len(x), -1))
        return y[-1]

#### Train

In [16]:
from tqdm import tqdm

def train_lstm(model, x, y, epochs=200, learning_rate=0.001):
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    
    
    for e in tqdm(range(epochs), desc="train_lstm"):
        for t in range(x.shape[0]):
            optimizer.zero_grad()
            
            model.hidden_cell = (torch.zeros(1, 1, model.hidden_dim).to(device),
                            torch.zeros(1, 1, model.hidden_dim).to(device))
        
            x_t = x[t]
            # to tensor
            x_t = torch.from_numpy(x[t]).float().to(device)
            
            y_t = y[t]
            y_t = torch.from_numpy(y[t]).float().to(device)
    
            out = model(x_t)
            loss = loss_fn(out, y_t)
    
            loss.backward()
            optimizer.step()
        
    
        if e % 50 == 0:
            print ("Epoch [{}/{}], Loss: {:.4f}".format(e + 1, epochs, loss.item()))

##### Train for Winter

In [17]:
winter_lstm = GrundwasserLSTM()
winter_lstm = winter_lstm.to(device)
print(winter_lstm)

GrundwasserLSTM(
  (lstm): LSTM(1, 100)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)


In [18]:
%time train_lstm(model=winter_lstm, x=x_train_winter, y=y_train_winter)

train_lstm:   0%|▎                                                                     | 1/200 [00:00<01:10,  2.81it/s]

Epoch [1/200], Loss: 5713.6377


train_lstm:  26%|█████████████████▌                                                   | 51/200 [00:18<01:01,  2.44it/s]

Epoch [51/200], Loss: 2.3166


train_lstm:  50%|██████████████████████████████████▎                                 | 101/200 [00:39<00:52,  1.89it/s]

Epoch [101/200], Loss: 4.5294


train_lstm:  76%|███████████████████████████████████████████████████▎                | 151/200 [01:00<00:21,  2.25it/s]

Epoch [151/200], Loss: 0.0789


train_lstm: 100%|████████████████████████████████████████████████████████████████████| 200/200 [01:26<00:00,  2.32it/s]

Wall time: 1min 26s


#### Infer

In [19]:
def predict(x, model):
    results = list()
    with torch.no_grad():
        for t in range(x.shape[0]):
            x_t = torch.from_numpy(x[t]).float().to(device)
            # it's a tensor, convert to numpy again
            predicted = model(x_t)
            results.append(predicted.cpu().detach().numpy())
            
    return results

In [20]:
y_pred = predict(x_test_winter, winter_lstm)

#### Evaluate

In [47]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

In [22]:
r2 = r2_score(y_pred=y_pred, y_true=y_test_winter)

In [23]:
r2

0.8385371092190064

In [48]:
mse = mse(y_pred=y_pred, y_true=y_test_winter)

In [50]:
mse

3.0061799689668582

In [24]:
mae = mae(y_pred=y_pred, y_true=y_test_winter)

In [25]:
mae

1.2754639973042805

##### Train for Summer

In [26]:
summer_lstm = GrundwasserLSTM()
summer_lstm = summer_lstm.to(device)
print(summer_lstm)

GrundwasserLSTM(
  (lstm): LSTM(1, 100)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)


In [27]:
%time train_lstm(model=summer_lstm, x=x_train_summer, y=y_train_summer)

train_lstm:   0%|▎                                                                     | 1/200 [00:00<00:55,  3.56it/s]

Epoch [1/200], Loss: 1877.3406


train_lstm:  26%|█████████████████▌                                                   | 51/200 [00:15<00:48,  3.07it/s]

Epoch [51/200], Loss: 238.5326


train_lstm:  50%|██████████████████████████████████▎                                 | 101/200 [00:33<00:42,  2.32it/s]

Epoch [101/200], Loss: 220.8226


train_lstm:  76%|███████████████████████████████████████████████████▎                | 151/200 [00:53<00:17,  2.85it/s]

Epoch [151/200], Loss: 37.9495


train_lstm: 100%|████████████████████████████████████████████████████████████████████| 200/200 [01:11<00:00,  2.81it/s]

Wall time: 1min 11s


In [28]:
y_pred_2 = predict(x_test_summer, summer_lstm)

### Save the trained model

In [29]:
if not os.path.exists(os.path.join(os.getcwd(), "saved_models")):
    os.mkdir(os.path.join(os.getcwd(), "saved_models"))


torch.save(winter_lstm.state_dict(), "./saved_models/winter")
torch.save(summer_lstm.state_dict(), "./saved_models/summer")

#### Preparing prediction model for Winter Data

In [30]:
def predict_winter(x, model):
    predicted_w = []
    for year in range(6):
        pred = predict(x, model)
        np.append(x, pred)
        predicted_w.append(pred[0])
    return predicted_w

#### Preparing prediction model for Summer Data

In [31]:
def predict_summer(x, model):
    predicted_s = []
    for year in range(5):
        pred = predict(x, model)
        np.append(x, pred)
        predicted_s.append(pred[0])
    return predicted_s

## Creating a dataframe for winter prediction


In [32]:
winter_count = winter_df.shape[0]

In [33]:
winter_count

95

In [34]:
def winter_pred():
    wp = dict()
    for i in range (winter_count):
        row = winter_df.iloc[i]
        station_name = row["Station"]
        row = row.drop(['Station','X','Y'])
        row = np.array([row])
        predict = predict_winter(row, winter_lstm)
        
        wp[station_name] = predict
    return wp

In [35]:
wp = winter_pred()

In [36]:
df_w = pd.DataFrame(list(wp.items()))

In [37]:
df_w

,0,1
0,25470023.0,"[[80.28582], [80.26513], [80.237625], [80.2115..."
1,25480025.0,"[[83.593254], [83.59252], [83.581406], [83.571..."
2,25480026.0,"[[81.56782], [81.65222], [81.64732], [81.64104..."
3,25481088.0,"[[76.25018], [76.23112], [76.20968], [76.18886..."
4,25490020.0,"[[68.22003], [68.241585], [68.25457], [68.2672..."
...,...,...
90,30491510.0,"[[82.699585], [82.697945], [82.696144], [82.69..."
91,30491512.0,"[[81.20746], [81.20567], [81.20407], [81.20254..."
92,30491513.0,"[[81.72599], [81.72321], [81.72045], [81.71787..."
93,30509090.0,"[[80.73162], [82.491295], [82.565926], [82.584..."


In [38]:
wp

{25470023.0: [array([80.28582], dtype=float32),
  array([80.26513], dtype=float32),
  array([80.237625], dtype=float32),
  array([80.211555], dtype=float32),
  array([80.18684], dtype=float32),
  array([80.1634], dtype=float32)],
 25480025.0: [array([83.593254], dtype=float32),
  array([83.59252], dtype=float32),
  array([83.581406], dtype=float32),
  array([83.57122], dtype=float32),
  array([83.56108], dtype=float32),
  array([83.55109], dtype=float32)],
 25480026.0: [array([81.56782], dtype=float32),
  array([81.65222], dtype=float32),
  array([81.64732], dtype=float32),
  array([81.641045], dtype=float32),
  array([81.63485], dtype=float32),
  array([81.628815], dtype=float32)],
 25481088.0: [array([76.25018], dtype=float32),
  array([76.23112], dtype=float32),
  array([76.20968], dtype=float32),
  array([76.18886], dtype=float32),
  array([76.1688], dtype=float32),
  array([76.1495], dtype=float32)],
 25490020.0: [array([68.22003], dtype=float32),
  array([68.241585], dtype=float3

In [80]:
df_w.to_csv("./Seasons/winter_pred.csv")

## Creating a dataframe for summer prediction

In [39]:
summer_count = summer_df.shape[0]

In [40]:
summer_count

95

In [41]:
def summer_pred():
    sp = dict()
    for i in range (summer_count):
        row = summer_df.iloc[i]
        station_name = row["Station"]
        row = row.drop(['Station','X','Y'])
        row = np.array([row])
        predict = predict_summer(row, winter_lstm)
        
        sp[station_name] = predict
    return sp

In [42]:
sp = summer_pred()

In [43]:
sp

{25470023: [array([43.162106], dtype=float32),
  array([43.160065], dtype=float32),
  array([43.172047], dtype=float32),
  array([43.184635], dtype=float32),
  array([43.19688], dtype=float32)],
 25480025: [array([69.88874], dtype=float32),
  array([69.84257], dtype=float32),
  array([69.83043], dtype=float32),
  array([69.818924], dtype=float32),
  array([69.8079], dtype=float32)],
 25480026: [array([66.68058], dtype=float32),
  array([66.680466], dtype=float32),
  array([66.68281], dtype=float32),
  array([66.68508], dtype=float32),
  array([66.687294], dtype=float32)],
 25481088: [array([65.831505], dtype=float32),
  array([65.94598], dtype=float32),
  array([65.985504], dtype=float32),
  array([65.99791], dtype=float32),
  array([66.00131], dtype=float32)],
 25490020: [array([51.21232], dtype=float32),
  array([51.649845], dtype=float32),
  array([51.606037], dtype=float32),
  array([51.57925], dtype=float32),
  array([51.558228], dtype=float32)],
 25500006: [array([38.327038], dty